New York Times Hardcover Fiction Best Sellers:
Looking at the performance of Authors and their Books
1. Import the JSON File

In [0]:
ny = spark.read.json("/FileStore/tables/nyt2.json")
ny.show(truncate = 30)

+--------------------------+------------------------------+------------------------------+-----------------+------------------------------+-------------+-----------------+-----------------------------+----+--------------+------------------------------+-------------+
|                       _id|            amazon_product_url|                        author| bestsellers_date|                   description|        price|   published_date|                    publisher|rank|rank_last_week|                         title|weeks_on_list|
+--------------------------+------------------------------+------------------------------+-----------------+------------------------------+-------------+-----------------+-----------------------------+----+--------------+------------------------------+-------------+
|{5b4aa4ead3089013507db18b}|http://www.amazon.com/Odd-H...|                 Dean R Koontz|{{1211587200000}}|Odd Thomas, who can communi...|   {null, 27}|{{1212883200000}}|                       Banta

Data Cleaning:

1. Find the Schema of the JSON file and look for any null values in the Dataframe

In [0]:
ny.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- amazon_product_url: string (nullable = true)
 |-- author: string (nullable = true)
 |-- bestsellers_date: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price: struct (nullable = true)
 |    |-- $numberDouble: string (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- published_date: struct (nullable = true)
 |    |-- $date: struct (nullable = true)
 |    |    |-- $numberLong: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- rank: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- rank_last_week: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- title: string (nullable = true)
 |-- weeks_on_list: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)



Null Value count in each column:

In [0]:
from pyspark.sql.functions import col, sum
null_values = ny.select([sum(col(column).isNull().cast("int")).alias("Null Values/Column") for column in ny.columns])
null_values.show()

+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|Null Values/Column|Null Values/Column|Null Values/Column|Null Values/Column|Null Values/Column|Null Values/Column|Null Values/Column|Null Values/Column|Null Values/Column|Null Values/Column|Null Values/Column|Null Values/Column|
+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|                 0|                 0|                 0|                 0|                 0|                 0|                 0|                 0|                 0|                 0|                 0|                 0|
+------------------+------------------+------------------+------------------+---

Geting rid of the Amazon URL column as it is not needed.

In [0]:
ny = ny.drop("amazon_product_url")
ny.show(truncate = 12)

+------------+------------+----------------+------------+------------+--------------+------------+----+--------------+------------+-------------+----------------------------------+--------------------------------+
|         _id|      author|bestsellers_date| description|       price|published_date|   publisher|rank|rank_last_week|       title|weeks_on_list|bestsellers_date.$date.$numberLong|published_date.$date.$numberLong|
+------------+------------+----------------+------------+------------+--------------+------------+----+--------------+------------+-------------+----------------------------------+--------------------------------+
|{5b4aa4ea...|Dean R Ko...|    {{1211587...|Odd Thoma...|  {null, 27}|  {{1212883...|      Bantam| {1}|           {0}|   ODD HOURS|          {1}|                      03-09-+40363|                    27-09-+40404|
|{5b4aa4ea...|Stephenie...|    {{1211587...|Aliens ha...|{25.99, n...|  {{1212883...|Little, B...| {2}|           {1}|    THE HOST|          {3}

Data Conversion:
1. Converting bestsellers_date and published_date from their Unix Timestamps to a more readable Date format

In [0]:
from pyspark.sql.functions import from_unixtime, date_format, col

# converting unix timestamp to normal date format
columns = ["bestsellers_date.$date.$numberLong", "published_date.$date.$numberLong"]

# Unix timestamps to date format
for column in columns:
    nested_fields = column.split(".")
    field = nested_fields[-1]

    ny = ny.withColumn(column, date_format(
        from_unixtime(col(column) / 1000), 
        "dd-MM-yyyy"
    ))
    
ny_best = ny.drop("bestsellers_date", "published_date")
ny_best = ny_best.withColumnRenamed("bestsellers_date.$date.$numberLong", "bestsellers_date")\
          .withColumnRenamed("published_date.$date.$numberLong", "published_date")

ny_best.show(truncate = 16)

+----------------+----------------+----------------+-------------+----------------+----+--------------+----------------+-------------+----------------+--------------+
|             _id|          author|     description|        price|       publisher|rank|rank_last_week|           title|weeks_on_list|bestsellers_date|published_date|
+----------------+----------------+----------------+-------------+----------------+----+--------------+----------------+-------------+----------------+--------------+
|{5b4aa4ead308...|   Dean R Koontz|Odd Thomas, w...|   {null, 27}|          Bantam| {1}|           {0}|       ODD HOURS|          {1}|      24-05-2008|    08-06-2008|
|{5b4aa4ead308...| Stephenie Meyer|Aliens have t...|{25.99, null}|   Little, Brown| {2}|           {1}|        THE HOST|          {3}|      24-05-2008|    08-06-2008|
|{5b4aa4ead308...|    Emily Giffin|A woman's hap...|{24.95, null}|    St. Martin's| {3}|           {2}|LOVE THE ONE ...|          {2}|      24-05-2008|    08-06-2008

Data Querying: 

Authors and their most expensive book in descending order:

In [0]:
# from pyspark.sql.functions import desc
author_rank = ny.groupBy("author").agg({"price": "max"})
authors_rank = author_rank.orderBy(desc("max(price)"))

authors_rank.show()

+--------------------+-------------+
|              author|   max(price)|
+--------------------+-------------+
|Robert Jordan and...|{34.99, null}|
|     Haruki Murakami| {30.5, null}|
|      Terry Goodkind|{29.99, null}|
|           Dan Brown|{29.95, null}|
|     Neal Stephenson|{29.95, null}|
|          Pat Conroy|{29.95, null}|
|    Patrick Rothfuss|{29.95, null}|
| Stephen R Donaldson|{29.95, null}|
|          Wally Lamb|{29.95, null}|
|    Bernard Cornwell|{28.99, null}|
|  Barbara Kingsolver|{28.99, null}|
|    Elizabeth George|{28.99, null}|
|     James Patterson|{28.99, null}|
|        Jodi Picoult|{28.99, null}|
|Michael Crichton ...|{28.99, null}|
|Clive Cussler and...|{28.95, null}|
|    Deborah Harkness|{28.95, null}|
|        James Ellroy|{28.95, null}|
|        John Grisham|{28.95, null}|
|   Patricia Cornwell|{28.95, null}|
+--------------------+-------------+
only showing top 20 rows



Average book price by Publisher

In [0]:
from pyspark.sql.functions import avg, round

ny_avgprice = ny.groupBy("publisher").agg(round(avg("price.$numberDouble"), 2).alias("Average Price per Publisher"))
ny_avgprice = ny_avgprice.orderBy(desc("Average Price per Publisher"))

ny_avgprice.show()

+--------------------+---------------------------+
|           publisher|Average Price per Publisher|
+--------------------+---------------------------+
|     Tor/Tom Doherty|                      30.66|
|      Tor/TomDoherty|                      29.99|
|                 Tor|                      29.99|
|                 DAW|                      29.95|
|Nan A. Talese/Dou...|                       29.2|
|  Marion Wood/Putnam|                      28.95|
| Emily Bestler/Atria|                      28.24|
|HarperCollins Voy...|                      27.99|
|Little, Brown & C...|                      27.99|
|         St.Martin’s|                      27.99|
|Ecco/HarperCollin...|                      27.99|
|             Doherty|                      27.95|
|Wizards of the Coast|                      27.95|
|       Penguin Group|                      27.95|
|  Marian Wood/Putnam|                      27.84|
|              Harper|                      27.73|
|Thomas Dunne/St. ...|         

Authors who have had books in the Best Sellers list for more than 20 weeks:

In [0]:
ny_books = ny.select("author", "title", "publisher", "weeks_on_list")

# Filtering on the $numberInt data in the Column, dropping duplicates found when querying data.
ny_books = ny_books.filter(col("weeks_on_list.$numberInt") > 20)
ny_book = ny_books.dropDuplicates(["author"])
ny_book = ny_book.sort("weeks_on_list")

ny_book.show()

+-----------------+--------------------+--------------------+-------------+
|           author|               title|           publisher|weeks_on_list|
+-----------------+--------------------+--------------------+-------------+
|          AJ Finn|THE WOMAN IN THE ...|              Morrow|         {21}|
|      Amor Towles|A GENTLEMAN IN MO...|              Viking|         {21}|
|    Anthony Doerr|ALL THE LIGHT WE ...|            Scribner|         {21}|
|       Celeste Ng|LITTLE FIRES EVER...|       Penguin Press|         {21}|
| Charlaine Harris|       DEAD AND GONE|                 Ace|         {21}|
| Colson Whitehead|THE UNDERGROUND R...|           Doubleday|         {21}|
|        Dan Brown|     THE LOST SYMBOL|           Doubleday|         {21}|
| David Wroblewski|THE STORY OF EDGA...|                Ecco|         {21}|
|      Donna Tartt|       THE GOLDFINCH|       Little, Brown|         {21}|
|George R R Martin|A DANCE WITH DRAGONS|              Bantam|         {21}|
|    Gillian

How many Authors do Publishers have featuring in the Best Sellers list.

In [0]:
ny_publishers = ny.groupBy("publisher").agg({"author": "count"})
ny_publishers = ny_publishers.orderBy(desc("count(author)"))

ny_publishers.show()

+--------------------+-------------+
|           publisher|count(author)|
+--------------------+-------------+
|              Putnam|         1061|
|       Little, Brown|          943|
|       Grand Central|          619|
|               Knopf|          559|
|           Doubleday|          475|
|            Scribner|          456|
|          Ballantine|          421|
|           Delacorte|          347|
|              Bantam|          319|
|    Simon & Schuster|          308|
|        St. Martin's|          301|
|              Viking|          269|
|        Random House|          252|
|Morrow/HarperCollins|          240|
|              Dutton|          231|
|              Harper|          211|
|           Riverhead|          199|
|  Amy Einhorn/Putnam|          185|
|               Atria|          183|
|        St. Martin’s|          178|
+--------------------+-------------+
only showing top 20 rows



In [0]:
putnam_authors = ny_best.filter(ny_best["publisher"] == "Putnam")
putnam = putnam_authors.groupBy("author").agg({"author":"count"})
putnam = putnam.orderBy(desc("count(author)"))

putnam.show()

+--------------------+-------------+
|              author|count(author)|
+--------------------+-------------+
|        Stuart Woods|          107|
|       John Sandford|          107|
|            J D Robb|           57|
|   Patricia Cornwell|           50|
|        Nora Roberts|           48|
|Clive Cussler and...|           41|
|   Catherine Coulter|           33|
|     Robert B Parker|           31|
|W E B Griffin and...|           31|
|Tom Clancy with M...|           28|
|             C J Box|           27|
|Clive Cussler and...|           26|
|Clive Cussler and...|           26|
|           Jan Karon|           26|
|   Randy Wayne White|           24|
|        Mark Greaney|           24|
|        Robert Crais|           24|
|        Daniel Silva|           19|
|       Alex Berenson|           17|
|Clive Cussler wit...|           14|
+--------------------+-------------+
only showing top 20 rows



Conclusion:

The Publisher Putnam has had a lot of success with their authors, who have had over 1000 of them featured in the New York Bestsellers list, while also not publishing the most expensive books amongst other publishers as well. Looking at the authors who have had their books published by Putnam, Stuart Woods and John Sandford both share the most books published by them with 107 books each, sharing a total of 20.2% (214 / 1061) of the amount of books published by Putnam.